In [44]:
using QPSReader
using QuadraticModels
using NLPModels

In [39]:
ENV["XPRESSDIR"] = "C:\\xpressmp"


"C:\\xpressmp"

In [40]:
Pkg.build("Xpress")

   Building MbedTLS → `C:\Users\Geoffroy Leconte\.julia\packages\MbedTLS\a1JFn\deps\build.log`
   Building Xpress ─→ `C:\Users\Geoffroy Leconte\.julia\packages\Xpress\kcAft\deps\build.log`


In [41]:
using Xpress

┌ Info: Precompiling Xpress [9e70acf3-d6c9-5be6-b5bd-4e2c73e3e054]
└ @ Base loading.jl:1260
┌ Info: Xpress: Found license file xpauth.xpr
└ @ Xpress C:\Users\Geoffroy Leconte\.julia\packages\Xpress\kcAft\src\license.jl:42
┌ Info: Xpress: Development license detected.
└ @ Xpress C:\Users\Geoffroy Leconte\.julia\packages\Xpress\kcAft\src\license.jl:84


In [46]:
function createQuadraticModel(qpdata)
    # probleme du point initial
    #x0 = init_x0(qpdata.lvar, qpdata.uvar)
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
            Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals, 
            lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
            c0=qpdata.c0)

end

function display_results_Xpress(xpress_model)
    # show results
    sol = get_solution(gurobi_model)
    println("soln = $(sol)")

    objv = get_objval(gurobi_model)
    println("objv = $(objv)")
end
    

function OptimizeXpress(qpdata)
    QM = createQuadraticModel(qpdata)
    SM = SlackModel(QM)
    
    env = Gurobi.Env()

    # set presolve to 0
    setparam!(env, "Presolve", 0)
    
    model = gurobi_model(env; f = grad(SM, zeros(length(SM.meta.x0))),
                Aeq = jac(SM, SM.meta.x0), beq = SM.meta.lcon, 
                lb = SM.meta.lvar, ub = SM.meta.uvar)

     # run optimization
    optimize(model)
    return model
end


OptimizeXpress (generic function with 1 method)

In [47]:
path_pb = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib"
afiro = string(path_pb, "\\AFIRO.SIF")
qpdata2 = readqps(afiro);

┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [49]:
SM1 = SlackModel(createQuadraticModel(qpdata2));